In [1]:
from transformers import TFBertForSequenceClassification, BertTokenizer
import tensorflow as tf
import pandas as pd
import numpy as np
from data_preparation import Example, convert_examples_to_tf_dataset, make_batches
import os

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.load_weights()
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
max_length = 512
batch_size = 32

In [61]:
unknown = []

for token in " ".join(test["review"].values).split():
    if tokenizer.encode(token, add_special_tokens=False)[0] == 100 and token not in unknown:
        unknown.append(token)

In [62]:
unknown

['’', '–', '‘', '“', '”', '…']

In [2]:
test = pd.read_csv("../data/sentiment/" + "tr" + "/test.csv", header=None)
test.columns = ["sentiment", "review"]

In [60]:
tokenizer.encode("’", add_special_tokens=False)

[100]

In [63]:
tokenizer.decode([100], clean_up_tokenization_spaces=False)

'[UNK]'

In [40]:
example_ids = []
sentiments = []
reviews = []

for index, row in test.iterrows():
    tokenized_review = tokenizer.encode(row["review"], add_special_tokens=False)
        
    for i in range(0, len(tokenized_review), 510): # Leaving two tokens for [CLS] and [SEP]
        example_ids.append(index)
        sentiments.append(row["sentiment"])
        decoded_chunk = tokenizer.decode(tokenized_review[i:i+510], 
                                         skip_special_tokens=True, 
                                         clean_up_tokenization_spaces=False)
        if decoded_chunk.startswith("##"):
            decoded_chunk = decoded_chunk[2:] # Otherwise the ## will be treated as actual chars
        elif i != 0 and not decoded_chunk.startswith("##"):
            decoded_chunk = " " + decoded_chunk # Do not lose space
        reviews.append(decoded_chunk)
        
new_test = pd.DataFrame({"example_id": example_ids, "sentiment": sentiments, "review": reviews})

Token indices sequence length is longer than the specified maximum sequence length for this model (1608 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (965 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1495 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1177 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1292 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

Token indices sequence length is longer than the specified maximum sequence length for this model (1220 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1131 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1811 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1903 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1175 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

Token indices sequence length is longer than the specified maximum sequence length for this model (796 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1140 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (802 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1203 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1651 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 

Token indices sequence length is longer than the specified maximum sequence length for this model (1327 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1538 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1126 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1315 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (985 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

Token indices sequence length is longer than the specified maximum sequence length for this model (1337 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1082 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1014 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (992 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (859 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 

In [41]:
test.shape, new_test.shape

((184, 2), (537, 3))

In [46]:
test.loc[0, "review"]

'Sinemanın en inişli çıkışlı kariyerlerinden birine sahiptir M . Night Shyamalan . Hint asıllı yönetmen ve senaryo yazarı , 1999 yılında başrolünde Bruce Willis’in bulunduğu 6 . His filmiyle adını tüm dünyaya duyurmuş , ertesi sene daha olgun ve çekici bir yapım olan Ölümsüz ile tekrar kendine hayran bırakmıştı . Willis ise yine başroldeydi . 2001 ’ deki İşaretler filmiyle bir kez daha korku türüne dönüş yapan Shyamalan , müthiş olmasa bile eli yüzü düzgün , kaliteli bir işe imza attı . Bazılarının finalini yerden yere vurduğu ancak sürpriz sonu da dahil olmak üzere iyi bir film olarak değerlendirilen Köy’ün ardından , Hollywood sineması anlayışından uzaklaştı . Sudaki Kız doğa üstü bir dramdı ancak korkup , heyecanlanmak isteyen seyirci için hayal kırıklığına dönüştü . Mistik Olay özünde basit ama iyi bir fikre sahip , uygulamada yer yer aksayan ancak toplamında gişe filmi olmak istememiş bir projeydi ; pek beğenilmedi . Ünlü bir anime serisinden uyarlanan ve belki de M . Night Shyama

In [45]:
"".join(new_test[new_test["example_id"] == 0].loc[:,"review"].values)

'Sinemanın en inişli çıkışlı kariyerlerinden birine sahiptir M . Night Shyamalan . Hint asıllı yönetmen ve senaryo yazarı , 1999 yılında başrolünde Bruce Willis in bulunduğu 6 . His filmiyle adını tüm dünyaya duyurmuş , ertesi sene daha olgun ve çekici bir yapım olan Ölümsüz ile tekrar kendine hayran bırakmıştı . Willis ise yine başroldeydi . 2001 deki İşaretler filmiyle bir kez daha korku türüne dönüş yapan Shyamalan , müthiş olmasa bile eli yüzü düzgün , kaliteli bir işe imza attı . Bazılarının finalini yerden yere vurduğu ancak sürpriz sonu da dahil olmak üzere iyi bir film olarak değerlendirilen Köy ün ardından , Hollywood sineması anlayışından uzaklaştı . Sudaki Kız doğa üstü bir dramdı ancak korkup , heyecanlanmak isteyen seyirci için hayal kırıklığına dönüştü . Mistik Olay özünde basit ama iyi bir fikre sahip , uygulamada yer yer aksayan ancak toplamında gişe filmi olmak istememiş bir projeydi ; pek beğenilmedi . Ünlü bir anime serisinden uyarlanan ve belki de M . Night Shyamala

In [ ]:
path = "../data/sentiment/"
for directory in os.listdir(path):
    lang_path = os.path.join(path, directory)
    test = pd.read_csv(lang_path + "/test.csv", header=None)
    test.columns = ["sentiment", "review"]
    
    if test["review"].apply(lambda x: len(tokenizer(x)["input_ids"])).max() <= 512:
        test_dataset = convert_examples_to_tf_dataset([(Example(text=text, category_index=label)) for label, 
                                                       text in test.values], 
                                                      tokenizer, max_length=max_length)
        test_dataset, test_batches = make_batches(test_dataset, batch_size, repetitions=1)
        #### Evaluate here
        
    else: